In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('diabetes_prediction_dataset.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [ ]:
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


## Data cleaning:
- Removal of gender "other" and smoking "no info" due to lack of clarity on actual values (e.g. transgender, decline to state, etc).  Entries would likely not provide insight for feature correlation to category outcome.
- Conversion of gender and smoking history to numeric values using dictionary

In [ ]:
dataValDict = {'gender': {'Male': 0, 'Female': 1, 'Other': 3}, 'smoking_history': {'never': 0, 'current': 1, 'ever': 2, 'former': 3, 'not current': 4}}

#Original file is saved as original data.  Df will be 'cleaned' file.
df = data[data['gender'] != 'Other']
df = data[data['smoking_history'] != 'No Info']

df = df.replace(dataValDict)
df['gender'] = pd.to_numeric(df['gender'])
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 64184 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               64184 non-null  int64  
 1   age                  64184 non-null  float64
 2   hypertension         64184 non-null  int64  
 3   heart_disease        64184 non-null  int64  
 4   smoking_history      64184 non-null  int64  
 5   bmi                  64184 non-null  float64
 6   HbA1c_level          64184 non-null  float64
 7   blood_glucose_level  64184 non-null  int64  
 8   diabetes             64184 non-null  int64  
dtypes: float64(3), int64(6)
memory usage: 4.9 MB


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,1,80.0,0,1,0,25.19,6.6,140,0
2,0,28.0,0,0,0,27.32,5.7,158,0
3,1,36.0,0,0,1,23.45,5.0,155,0
4,0,76.0,1,1,1,20.14,4.8,155,0
5,1,20.0,0,0,0,27.32,6.6,85,0
...,...,...,...,...,...,...,...,...,...
99992,1,26.0,0,0,0,34.34,6.5,160,0
99993,1,40.0,0,0,0,40.69,3.5,155,0
99997,0,66.0,0,0,3,27.83,5.7,155,0
99998,1,24.0,0,0,0,35.42,4.0,100,0


## Training and Test set creation:

In [ ]:
#Split features from (diabetes) label

features = ['gender','age','hypertension','heart_disease','smoking_history', 'bmi','HbA1c_level','blood_glucose_level']

df_check_ensemble = df.sample(5000)
df = df.drop(df_check_ensemble)


X = df.loc[:, features]
y = df.loc[:, ['diabetes']]

X_check_ensemble = df_check_ensemble.loc[:, features]
y_check_ensemble = df_check_ensemble.loc[:, ['diabetes']]
#Data will be shuffled before split by default. No stratification. Test size = 30% of set.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .70)
X_train.shape, X_test.shape

((44928, 8), (19256, 8))

In [ ]:
#Create np array versions of test and train data
X_trn_np = X_train.to_numpy() 
X_tst_np = X_test.to_numpy() 
y_trn_np = y_train.to_numpy() 
y_tst_np = y_test.to_numpy() 

In [ ]:
from tensorflow import keras

activation_type = 'relu'

model = keras.Sequential([
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation=activation_type, kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation=activation_type),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(64, activation=activation_type),
    keras.layers.Dense(64, activation=activation_type),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(32, activation=activation_type),
    keras.layers.Dense(32, activation=activation_type),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(32, activation=activation_type, kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation=activation_type),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(16, activation=activation_type, kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(16, activation=activation_type),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(8, activation=activation_type, kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(8, activation=activation_type),
    keras.layers.Dense(1)
])
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=keras.losses.BinaryFocalCrossentropy(from_logits=True), metrics=['accuracy'])
callbacks = []
callbacks.append(keras.callbacks.EarlyStopping(monitor='val_loss', patience=25))
callbacks.append(keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5))
model.fit(X_train, y_train, epochs=250, batch_size=50, validation_split=0.1, callbacks=callbacks)

evaluation = model.evaluate(X_test, y_test, batch_size = 1)
print(evaluation[1])


Epoch 1/250
809/809 [==============================] - 4s 2ms/step - loss: 0.1986 - accuracy: 0.9172 - val_loss: 0.0408 - val_accuracy: 0.9526 - lr: 0.0010
Epoch 2/250
809/809 [==============================] - 2s 2ms/step - loss: 0.0472 - accuracy: 0.9366 - val_loss: 0.0343 - val_accuracy: 0.9570 - lr: 0.0010
Epoch 3/250
809/809 [==============================] - 2s 2ms/step - loss: 0.0439 - accuracy: 0.9387 - val_loss: 0.0353 - val_accuracy: 0.9490 - lr: 0.0010
Epoch 4/250
809/809 [==============================] - 2s 2ms/step - loss: 0.0422 - accuracy: 0.9421 - val_loss: 0.0339 - val_accuracy: 0.9548 - lr: 0.0010
Epoch 5/250
809/809 [==============================] - 2s 2ms/step - loss: 0.0420 - accuracy: 0.9427 - val_loss: 0.0337 - val_accuracy: 0.9557 - lr: 0.0010
Epoch 6/250
809/809 [==============================] - 2s 2ms/step - loss: 0.0408 - accuracy: 0.9439 - val_loss: 0.0341 - val_accuracy: 0.9555 - lr: 0.0010
Epoch 7/250
809/809 [==============================] - 2s 2ms/st

In [1]:
#model.save('DanielNeuralNetworkModel.keras')
loaded_model = keras.models.load_model('DanielNeuralNetworkModel.keras')

predictions = loaded_model.predict(([X_check_ensemble]))
probabilities = 1 / (1 + np.exp(-predictions))

print(probabilities)

NameError: name 'keras' is not defined